# **Mô hình SVM**

## **Import thư viện**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import os

## **Đọc dữ liệu**

In [2]:
df = pd.read_csv("../../data/data_processed/data_processed.csv")
df.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,4.0,0.0,4.0,2.0,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,2.0,0.0,5.0,2.0,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,2.0,0.0,6.0,2.0,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,4.0,0.0,5.0,0.0,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


## **Xây dựng mô hình**

In [3]:
def evaluate_model(X_train, X_test, y_train, y_test, kernel='rbf'):
    # model = SVC(kernel=kernel, random_state=42)
    model = SVC(kernel='rbf', class_weight='balanced', decision_function_shape='ovr')
    model.fit(X_train, y_train)
    
    print("[Train] Classification Report:")
    print(classification_report(y_train, model.predict(X_train)))
    
    print("[Test] Classification Report:")
    print(classification_report(y_test, model.predict(X_test)))

# Xử lý dữ liệu gốc theo tỉ lệ và áp dụng SVM 
def process_original_data(df, ratio, kernel='rbf'):
    X = df.drop('NSP', axis=1)
    y = df['NSP']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=1 - ratio, stratify=y, random_state=42
    )
    
    print(f"\nMô hình SVM với dữ liệu gốc - Tỉ lệ = {int(ratio*10)}:{int((1-ratio)*10)} ---")
    evaluate_model(X_train, X_test, y_train, y_test, kernel=kernel)


In [4]:
def process_presplit_data(train_path, test_path, target_col='Target', kernel='rbf', method='PCA'):
    # Đọc dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Tách X và y
    X_train = train_data.drop('Target', axis=1).values
    y_train = train_data['Target'].values

    X_test = test_data.drop('Target', axis=1).values
    y_test = test_data['Target'].values

    print(f"\nMô hình SVM với dữ liệu giảm chiều bằng {method} tỉ lệ {split}")


    # Huấn luyện và đánh giá
    model = SVC(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)

    print("[Train] Classification Report:")
    print(classification_report(y_train, model.predict(X_train)))

    print("[Test] Classification Report:")
    print(classification_report(y_test, model.predict(X_test)))


## **Dữ liệu gốc**

In [5]:
for r in [0.8, 0.7, 0.6]:  
    process_original_data(df, r)


Mô hình SVM với dữ liệu gốc - Tỉ lệ = 8:1 ---
[Train] Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      0.90      0.94      1323
         2.0       0.61      0.95      0.75       236
         3.0       0.91      0.94      0.93       141

    accuracy                           0.91      1700
   macro avg       0.84      0.93      0.87      1700
weighted avg       0.94      0.91      0.92      1700

[Test] Classification Report:
              precision    recall  f1-score   support

         1.0       0.99      0.88      0.93       332
         2.0       0.55      0.88      0.68        59
         3.0       0.78      0.80      0.79        35

    accuracy                           0.87       426
   macro avg       0.77      0.85      0.80       426
weighted avg       0.91      0.87      0.88       426


Mô hình SVM với dữ liệu gốc - Tỉ lệ = 7:3 ---
[Train] Classification Report:
              precision    recall  f1-score   suppor

## **Dữ liệu giảm chiều bằng PCA**

In [6]:
file_paths_pca = {
    "80_20": {
        'train': r'..\..\data\dimension_reduction\pca\train_80_20.csv',
        'test': r'..\..\data\dimension_reduction\pca\test_80_20.csv'
    },
    "70_30": {
        'train': r'..\..\data\dimension_reduction\pca\train_70_30.csv',
        'test': r'..\..\data\dimension_reduction\pca\test_70_30.csv'
    },
    "60_40": {
        'train': r'..\..\data\dimension_reduction\pca\train_60_40.csv',
        'test': r'..\..\data\dimension_reduction\pca\test_60_40.csv'
    }
}

for split, paths in file_paths_pca.items():
    process_presplit_data(
        train_path=paths['train'],
        test_path=paths['test'],
        target_col='Target',
        method=f'PCA'
    )



Mô hình SVM với dữ liệu giảm chiều bằng PCA tỉ lệ 80_20
[Train] Classification Report:
              precision    recall  f1-score   support

         1.0       0.95      0.97      0.96      1323
         2.0       0.76      0.75      0.75       236
         3.0       1.00      0.82      0.90       141

    accuracy                           0.93      1700
   macro avg       0.90      0.85      0.87      1700
weighted avg       0.93      0.93      0.93      1700

[Test] Classification Report:
              precision    recall  f1-score   support

         1.0       0.93      0.97      0.95       332
         2.0       0.67      0.63      0.65        59
         3.0       0.88      0.66      0.75        35

    accuracy                           0.90       426
   macro avg       0.83      0.75      0.78       426
weighted avg       0.89      0.90      0.89       426


Mô hình SVM với dữ liệu giảm chiều bằng PCA tỉ lệ 70_30
[Train] Classification Report:
              precision    recal

## **Dữ liệu giảm chiều bằng LDA**

In [7]:
file_paths_lda = {
    "80_20": {
        'train': r'..\..\data\dimension_reduction\lda\train_80_20.csv',
        'test': r'..\..\data\dimension_reduction\lda\test_80_20.csv'
    },
    "70_30": {
        'train': r'..\..\data\dimension_reduction\lda\train_70_30.csv',
        'test': r'..\..\data\dimension_reduction\lda\test_70_30.csv'
    },
    "60_40": {
        'train': r'..\..\data\dimension_reduction\lda\train_60_40.csv',
        'test': r'..\..\data\dimension_reduction\lda\test_60_40.csv'
    }
}

for split, paths in file_paths_lda.items():
    process_presplit_data(
        train_path=paths['train'],
        test_path=paths['test'],
        target_col='Target',
        method=f'LDA'
    )



Mô hình SVM với dữ liệu giảm chiều bằng LDA tỉ lệ 80_20
[Train] Classification Report:
              precision    recall  f1-score   support

         1.0       0.94      0.96      0.95      1323
         2.0       0.69      0.67      0.68       236
         3.0       0.93      0.76      0.84       141

    accuracy                           0.90      1700
   macro avg       0.85      0.80      0.82      1700
weighted avg       0.90      0.90      0.90      1700

[Test] Classification Report:
              precision    recall  f1-score   support

         1.0       0.95      0.96      0.95       332
         2.0       0.67      0.71      0.69        59
         3.0       0.85      0.66      0.74        35

    accuracy                           0.90       426
   macro avg       0.82      0.78      0.79       426
weighted avg       0.90      0.90      0.90       426


Mô hình SVM với dữ liệu giảm chiều bằng LDA tỉ lệ 70_30
[Train] Classification Report:
              precision    recal

### 1. **PCA có thể giúp SVM hoạt động hiệu quả hơn trong không gian ít nhiễu**
- **PCA** tìm chiều không gian mới sao cho **giữ lại phương sai lớn nhất**, giúp loại bỏ nhiễu và các chiều không đóng góp nhiều thông tin. Điều này đặc biệt hữu ích cho các mô hình **nhạy với nhiễu** như SVM.
- Khi dữ liệu gốc chứa nhiều chiều dư thừa (high-dimensional noise), SVM trên dữ liệu gốc sẽ **khó tìm biên phân cách tối ưu** => giảm chính xác.

###  2. **LDA có thể overfit nếu số chiều sau giảm thấp hoặc phân bố không đều**
- LDA tối ưu hóa việc **phân tách các lớp** thông qua ma trận giữa và trong lớp, nhưng nếu dữ liệu không phân tách tuyến tính rõ ràng, hoặc số mẫu của mỗi lớp chênh lệch lớn → **LDA không mạnh bằng PCA**.
- LDA giới hạn số chiều sau giảm = `số lớp - 1` (ví dụ có 3 lớp thì tối đa còn 2 chiều) → mất thông tin → hạn chế khả năng biểu diễn cho SVM (nhất là kernel SVM).

### 3. **Kernel SVM + PCA = kết hợp tốt**
- SVM dùng kernel `rbf` (phi tuyến) có thể **học được ranh giới phi tuyến phức tạp**. PCA giữ được thông tin quan trọng → giúp kernel hoạt động hiệu quả hơn.
- Trong khi đó, nếu đầu vào chỉ còn 2 chiều (như LDA với 3 lớp), **biên phân cách trong không gian giảm** có thể không đủ linh hoạt cho SVM.


### Tóm lại:

| Mô hình | Ưu điểm | Nhược điểm |
|--------|---------|-------------|
| **Gốc (raw)** | Đầy đủ thông tin | Dễ nhiễu, overfit |
| **PCA** | Giảm nhiễu, giữ phương sai | Không phân biệt lớp |
| **LDA** | Phân biệt lớp tốt | Mất thông tin nếu lớp lệch, chỉ còn `n_classes - 1` chiều |

